# Camera Translation from Centroids 
This script, given all the (assumed) near-correct location, try to use known camera matrix to calculate the translation between training and test pictures.

In [1]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from kmeans import displacement_calculation

## Reading in data

In [2]:
train_xy = pd.read_csv('train.csv', index_col=0)
train_path = train_xy.index
test_path = pd.read_csv('imagenames.csv')['id'].values

good = pd.read_csv('./sift_matches_distance.csv').values
fgood = pd.read_csv('./sift_matches_filename.csv').values

#cluster_centroids = pd.read_csv('Results/best_cluster.csv')
cluster_centroids = pd.read_csv('Results/COCK_SIFT_voted_5_5_5_15_1_04_imputed.csv')

## Helper Function & Variables

In [3]:
def get_center_best(i, center, radius, max_count):
    '''Get the top SIFT matched results that is near the center

    Params
    ---
    - i: index of test instance
    - center: test instance's chosen center
    - radius: cluster radius
    - max_count: number of top instaces to take

    Returns
    ---
    - cluster_filenames: filenames of the cluster elements
    - cluster_locations: locations of the cluster elements
    '''
    count = 0
    cluster_fs = []
    cluster_locs = []
    if fgood[i,0] is np.nan: return cluster_fs, cluster_locs
    for tr in fgood[i]:
        if count == max_count: break
        train_loc = train_xy.loc[tr].values
        if np.sum((train_loc-center)**2)**0.5 < radius:
            cluster_fs.append(tr)
            cluster_locs.append(train_loc)
            count += 1
    return cluster_fs, cluster_locs

In [7]:
### Camera matrix, given:

# Image field of view
FOV_X = 73.3*np.pi/180
FOV_Y = 53.1*np.pi/180

# Pixel length
cx = 680/2
cy = 490/2

# Focal length in pixels
fx = cx/np.tan(FOV_X/2)
fy = cy/np.tan(FOV_Y/2)

# Complete intrinsic camera matrix
K = np.array([[fx,0,cx],
              [0,fy,cy],
              [0,0,1]])

## Run and export the data

In [8]:
radius = 10
max_count = 10
ratio = 0.7
max_range = 1.5
max_displacement = 10

res = []
for i,test in enumerate(tqdm(test_path)):
    # Getting the center and image
    center = cluster_centroids.iloc[i][['x','y']].values
    test_img = cv2.imread('./test/' + test_path[i] + '.jpg')
    # Getting the surrounding images
    cluster_fs, cluster_locs = get_center_best(i, center, radius, max_count)

    # Getting the final calculated location
    final_loc = displacement_calculation(test_img, center, 
                                         cluster_locs, cluster_fs, 
                                         K, ratio, max_range, max_displacement)
    res.append(final_loc)

100%|██████████| 1200/1200 [35:29<00:00,  1.77s/it]


In [9]:
out = pd.DataFrame(res,index=test_path)
out.to_csv('Results/SIFT_10_10_07_1p5_10.csv',index_label='id',header=['x','y'])